In [172]:
import pandas as pd
import warnings
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
from stargazer.stargazer import Stargazer
from sklearn.linear_model import LinearRegression, LogisticRegression

In [2]:
data = pd.read_csv('kickers_v2.csv')

In [61]:
data.head()

,Unnamed: 0,Team,Year,GameMinute,Kicker,Distance,ScoreDiff,Grass,Success,Intercept
0,1,PHI,2005,3,Akers,49,0,False,0,1
1,2,PHI,2005,29,Akers,49,-7,False,0,1
2,3,PHI,2005,51,Akers,44,-7,False,1,1
3,4,PHI,2005,14,Akers,43,14,True,0,1
4,5,PHI,2005,60,Akers,23,0,True,1,1


In [62]:
# It's important we check for NAN before we start our analysis.
data.isnull().values.any()

False

<h1> PSET 1 Econ 1042 Sports Economics </h1>
<h2> 1. Question </h2>
<ol>
    <li> What was the minimum distance of a field goal kicked in this sample? What was the maximum? Mean? Median!</li>
    <li> Why isn’t the minimum lower? (For those who are not familiar with football, please read about how field goal distance is measured and its relationship to where the ball is on the field.)</li>
    <li> What special circumstances might explain the maximum? (Hint: football is a game with 4, 15-minute quarters. At the end of the second quarter there is a halftime break and possession is assigned based on the result of a first-half coin toss) </li>
</ol>

In [14]:
print(f"The median distance of a field goal kicked was {np.median(data['Distance'])}")
data['Distance'].describe()

The median distance of a field goal kicked was 37.0


count    11187.000000
mean        36.897381
std         10.173351
min         18.000000
25%         28.000000
50%         37.000000
75%         45.000000
max         76.000000
Name: Distance, dtype: float64

1. The minimum distance of a field goal kicked in the sample was 18.00 yards. The maximum was 76.00 yards and the median was 37.0 yards. The mean was 36.897 yards
2. The minimum is 17 yards. This makes sense since the endzone is 10yards, and the ball has to be kicked from 7 yards from the line of scrimmage. Hence 10 + 7 = 17.

In [16]:
# Lets find out what play was kicked from 76 yards away?
max_yard = data.loc[data['Distance'] == 76.00]
max_yard

,Unnamed: 0,Team,Year,GameMinute,Kicker,Distance,ScoreDiff,Grass,Success
3557,3558,OAK,2008,30,Janikowski,76,15,True,0


3. The 76 yards field goal attempt from Janikowski was in the last second of the second quarter (video: https://www.youtube.com/watch?v=X7BepDe6Zoc). It makes sense to kick if far into the opponents end zone, if in the first half your team had the ball. Since, then the opposing team will start from further away from the kickers endzone. It's like as if the special team does a punt.

<h2> 2. Question </h2>
<p> Over the entire sample what percentage of kicks from 40 to 45 yards were made? Kicks over 45 yards? <p>

In [46]:
sample_size = len(data)
print(sample_size)
# let's find the number of successful kicks from 40-45 yards
kicks_40_45 = data.loc[(data['Distance'] > 40) & (data['Distance'] < 45)]
print(len(kicks_40_45))
# We find that 1325 kicks were made in that range
success_40_45 = kicks_40_45['Success'].value_counts()
# print(success_40_45)
ratio_success = success_40_45[1]/(success_40_45[0] + success_40_45[1])
ratio = (len(kicks_40_45)/sample_size) * 100
print(f'{ratio:.3f}% of Kicks were from between 40-45 yards')

# How many kicks were over 45 yards?
# kicks

11187
1325
11.844% of Kicks were from between 40-45 yards


In [49]:
kicks_above_45 = data.loc[data['Distance'] > 45]
ratio_above_45 = (len(kicks_above_45)/sample_size) * 100
print(f'{ratio_above_45:.3f}% of Kicks were from between 40-45 yards')

24.439% of Kicks were from between 40-45 yards


<h2> 3. Question </h2>
<p> Was the make rate higher on grass or on turf? Is that difference statistically significant? Do you think this is the true effect of surface? Why or why not?  (Answer this by doing an OLS regression. For the entire assignment, let’s use the heteroskedasticity robust standard errors, r in stata or the equivalent in R)<br> <br>
Let's compute the difference using $\Delta = \bar{Y}_{grass} - \bar{Y}_{turf}$ we shall report standard errors as heteroscedasticity robust (HC2) 
</p>

In [67]:
# define response variable
# statsmodel requires us to add a column where each value is 1 in order to compute intercept
data['Intercept'] = 1
# Since we find no NAN in our column
print(data['Grass'].isnull().values.any())
# We can conver the 'bool' values for Grass to the datatype 'int'
data['Grass'] = data['Grass'].astype(int)
Y = data[['Success']]
X = data[['Grass', 'Intercept']]
mod = sm.OLS(Y, X)
res = mod.fit(cov_type='HC2')
print(res.summary())

False
                            OLS Regression Results                            
Dep. Variable:                Success   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     7.500
Date:                Sun, 29 Jan 2023   Prob (F-statistic):            0.00618
Time:                        18:29:31   Log-Likelihood:                -4845.9
No. Observations:               11187   AIC:                             9696.
Df Residuals:                   11185   BIC:                             9710.
Df Model:                           1                                         
Covariance Type:                  HC2                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Grass         -0.0193      0.007     -2.739   

In [74]:
# Let's also try the Sklearn Library
# We can also use the Sklearn Library to do an OLS regression, but I don't think it has a summary function.
X = data[['Grass']]
reg = LinearRegression(fit_intercept=True).fit(X,Y)
parameters = reg.get_params()
print(reg.coef_)

[[-0.01932925]]


We find that the observed difference is statistically insignificant at the $\alpha = 0.05$ level. It seems as if the surface does not have an impact on the observed average success rates of field goal kicks.

<h2> 4. Question </h2>
<ol>
    <li>	How is distance of attempt correlated with surface? What might explain this? (Coaches get to choose when to kick a field goal, one is never forced) </li>
    <li> 	How is distance correlated with make percentage? </li>
</ol>

In [82]:
# Let's calculate the correlation between the columns
corr_surface = data['Distance'].corr(data['Grass'])
corr_success = data['Distance'].corr(data['Success'])
print(corr_surface)
print(corr_success)

-0.002551996001227438
-0.33693399701495164


The correlation coefficient for distance and or Grass is -0.0025, basically negligible. The correlation coefficiecnt for distance and success rates is -0.3369, meaning as distance increases the success rate goes down. 

<h2> 5. Question </h2>
<ol>
    <li>What is the formula for omitted variable bias?</li>
    <li><strong>Given (a) what should happen to the estimate of the effect of a kick being on grass when you add in distance? Verify this is true.</li>
</ol>

1. Ommitted variable bias arises when the regressor X is correlated with an omitted variable.
The formula for omitted variable bias is <br>$\hat{\beta}_{1,OLS}= \frac{Cov(y,x_1)}{Var(x_1)}$
2. We can estimate the effect of the ommited bias by adding in distance. Since, success rate and grass type are uncorrelated, we should be able to see <strong> NO IDEA.

In [98]:
Y = data[['Success']]
X = data[['Grass', 'Distance', 'Intercept']]
mod = sm.OLS(Y, X)
res = mod.fit(cov_type='HC2')
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                Success   R-squared:                       0.114
Model:                            OLS   Adj. R-squared:                  0.114
Method:                 Least Squares   F-statistic:                     695.7
Date:                Mon, 30 Jan 2023   Prob (F-statistic):          1.74e-285
Time:                        13:08:40   Log-Likelihood:                -4171.1
No. Observations:               11187   AIC:                             8348.
Df Residuals:                   11184   BIC:                             8370.
Df Model:                           2                                         
Covariance Type:                  HC2                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Grass         -0.0200      0.007     -3.004      0.0

In [97]:
mod = smf.ols(formula='Success ~ Grass + Distance', data=data).fit(cov_type='HC2')
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:                Success   R-squared:                       0.114
Model:                            OLS   Adj. R-squared:                  0.114
Method:                 Least Squares   F-statistic:                     695.7
Date:                Mon, 30 Jan 2023   Prob (F-statistic):          1.74e-285
Time:                        13:08:37   Log-Likelihood:                -4171.1
No. Observations:               11187   AIC:                             8348.
Df Residuals:                   11184   BIC:                             8370.
Df Model:                           2                                         
Covariance Type:                  HC2                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.2999      0.011    115.413      0.0

In [99]:
data.columns

Index(['Unnamed: 0', 'Team', 'Year', 'GameMinute', 'Kicker', 'Distance',
       'ScoreDiff', 'Grass', 'Success', 'Intercept'],
      dtype='object')

<h2> 6. Question </h2>
<ol>
    <li> Run an ols regression of kick success on distance, surface, point differential, and clock time. Interpret the coefficients. Does it seem like kickers do better or worse late in the game? Does the score of the game seem to effect them?</li>

<li>Now add in kicker fixed effects (i.kicker in Stata), what do these correct for? How does adjusted r-squared change? </li>
</ol>

In [102]:
mod = smf.ols(formula='Success ~ Distance + Grass + ScoreDiff + GameMinute', data=data).fit(cov_type='HC2')
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:                Success   R-squared:                       0.114
Model:                            OLS   Adj. R-squared:                  0.114
Method:                 Least Squares   F-statistic:                     347.8
Date:                Mon, 30 Jan 2023   Prob (F-statistic):          1.07e-282
Time:                        13:25:48   Log-Likelihood:                -4171.0
No. Observations:               11187   AIC:                             8352.
Df Residuals:                   11182   BIC:                             8389.
Df Model:                           4                                         
Covariance Type:                  HC2                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.2987      0.013    101.462      0.0

<p>1. The Game minute seems to have no statistically significant effect on Kicker performance. Furthermore, the point differential of the game does not seem to be statistically significant either, and also not affecting kicker success rates.</p>

In [103]:
mod = smf.ols(formula='Success ~ Distance + Grass + ScoreDiff + GameMinute + C(Kicker)', data=data).fit(cov_type='HC2')
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:                Success   R-squared:                       0.127
Model:                            OLS   Adj. R-squared:                  0.121
Method:                 Least Squares   F-statistic:                 8.581e+10
Date:                Mon, 30 Jan 2023   Prob (F-statistic):               0.00
Time:                        13:30:56   Log-Likelihood:                -4088.0
No. Observations:               11187   AIC:                             8350.
Df Residuals:                   11100   BIC:                             8987.
Df Model:                          86                                         
Covariance Type:                  HC2                                         
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                 

/opt/homebrew/Caskroom/miniforge/base/envs/deep/lib/python3.10/site-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 86, but rank is 85
  warnings.warn('covariance of constraints does not have full '


<p> 2. Adjusted R-square goes up, since we adjusting our regression for individuals attributes that do not vary over the time of a season(sample size).

<h2> 7. Question </h2>
<ol>
    <li> After you run the regression in part 6, do the command to predict fitted values from this regression: “predict, xb” in stata and equivalent in R “predict.lm”. Based on this, what would you predict the probability of Justin Tucker cutting the lead to 8 (scorediff was -11) in 2015, when the gameminute was 30, and he was on turf</li>
    <ol>
        <li> We can't build a model using all inputs, and then try to predict sth when missing one value. Missing here is an input for the <strong>Distance</li>
    </ol>

<li>Does this estimate strike you as reasonable?</li>
<li>What would the estimate be for an average kicker?</li>
</ol>

In [223]:
# Let's use the Sklearn library for this prediction. Since we can specify the input.
# Since we are interested in Tucker
data_tucker = data.loc[data['Kicker'] == 'Tucker']
X = data_tucker[['Distance', 'Grass', 'ScoreDiff', 'GameMinute']]
Y = data_tucker[['Success']]
reg = LinearRegression(fit_intercept=True).fit(X,Y)
parameters = reg.get_params()
# Let's find the distance of the kick from tucker with scroediff -11, gameminute 30, and on turf
tucker = data_tucker.loc[(data_tucker['ScoreDiff']==-11)&(data_tucker['GameMinute']==30) & (data_tucker['Grass']==False)]
tucker

reg_coef = list(np.ravel(reg.coef_))
reg_coef.insert(0, 'Linear_Tucker')
proba = reg.predict([[30, False, -11, 30]])[0][0]
reg_coef.append(proba)
print(reg_coef)
results = pd.DataFrame([reg_coef], columns=['Model', 'Distance', 'Grass', 'ScoreDiff', 'GameMinute', 'Proba'])
results

['Linear_Tucker', -0.012015139013776488, 0.09939115007000607, 0.0008766873127663746, 0.0007538568196675142, 0.9350746758606505]


/opt/homebrew/Caskroom/miniforge/base/envs/deep/lib/python3.10/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


,Model,Distance,Grass,ScoreDiff,GameMinute,Proba
0,Linear_Tucker,-0.012015,0.099391,0.000877,0.000754,0.935075


In [222]:
# This is the play that the question is asking for
# We can ignore the warning message
tucker = [[30, False, -11, 30]]
print(reg.predict(tucker)[0][0])

0.9350746758606505


/opt/homebrew/Caskroom/miniforge/base/envs/deep/lib/python3.10/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


2. Let's see if this estimate seems reasonable for Tucker

In [149]:
tucker_35_yards = data_tucker.loc[(data_tucker['Distance'] < 35)]
tucker_35_yards['Success'].value_counts()

1    61
Name: Success, dtype: int64

In the sample Tucker made $61$ kicks that were below 35 yards and he made every single one of them. Hence, the estimate in 1.) seems reasonable. On average in the sample we have $4765$ observations of Kicks that were shot at less then 35 yard distance. Of those $0.947\%$ were successful, hence the predicted probability seems to be a reasonable estimate.

In [154]:
data_35_yards = data.loc[data['Distance'] < 35]
data_35_yards['Success'].value_counts()

1    4513
0     252
Name: Success, dtype: int64

3. Let's predict the probability for an average kicker for that same shot.

In [219]:
X = data[['Distance', 'Grass', 'ScoreDiff', 'GameMinute']]
Y = data[['Success']]
reg = LinearRegression(fit_intercept=True).fit(X,Y)
parameters = reg.get_params()
reg_coef = list(np.ravel(reg.coef_))
reg_coef.insert(0, 'Linear')
proba = reg.predict([[30, False, -11, 30]])[0][0]
reg_coef.append(proba)
print(reg_coef)
results = pd.DataFrame([reg_coef], columns=['Model', 'Distance', 'Grass', 'ScoreDiff', 'GameMinute', 'Proba'])
results

['Linear', -0.012371812687093085, -0.01997241577141623, -0.0001022378401408906, 4.407522836155251e-05, 0.9300106512280658]


/opt/homebrew/Caskroom/miniforge/base/envs/deep/lib/python3.10/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


,Model,Distance,Grass,ScoreDiff,GameMinute,Proba
0,Linear,-0.012372,-0.019972,-0.000102,0.000044,0.930011


We estimate the probability for an average kicker to be 0.93%

<hr>
<h2> 8. Question </h2>
<ol>
    <li>Now run a logistic regression with the same specification as in question 6. Use the command predict. Now what is the predicted probability of Tucker making that field goal? (the predict command in stata is now just “predict”)</li>
    <li>Why do the coefficients look so different for the logistic regression vs. OLS </li>
</ol>

In [215]:
X = data[['Distance', 'Grass', 'ScoreDiff', 'GameMinute']]
Y = data['Success']
model_log = LogisticRegression(fit_intercept=True).fit(X,Y)
parameters = model_log.get_params()
model_log.predict_proba([[30, False, -11, 30]])
model_log_coef = list(np.ravel(model_log.coef_))
model_log_coef.insert(0, 'Logistic')
print(model_log_coef)
frame = pd.DataFrame(model_log_coef, columns=['Distance', 'Grass', 'ScoreDiff', 'GameMinute'])
frame

['Logistic', -0.10284160933932229, -0.16762221135876101, -0.0009622082796419989, 0.0003842398038772469]


/opt/homebrew/Caskroom/miniforge/base/envs/deep/lib/python3.10/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


ValueError: Shape of passed values is (5, 1), indices imply (5, 4)